<a href = '#Basics'>1. Basics</a><br>
<a href = '#LoadData'>1. a. Load Data</a><br>
<a href = '#DataTypes'>1. b. Data Types</a><br>
<br>
<a href = '#Plots'>2. Plots</a><br>
<a href = '#Univariate'>2.a. Univariate</a><br>
<a href = '#Bivariate'>2.b. Bivariate</a><br>
<br>
<a href = '#PredictiveModelingStarts'>3. Predictive Modeling Starts</a><br>
<a href = '#Logistic'>3.a. 1st model - Logistic</a><br>
<a href = '#RandomForest'>3.b. Random Forest</a><br>
<br>
<a href = '#OverVsUnderFitting'>4. Over Vs Underfitting</a><br>
<a href = '#BiasVsVariancec'>4.a. Bias Vs Variance</a><br>
<a href = '#ValidationSet'>4.b. Validation Set</a><br>
<br>
<br>
<a href = '#EvaluationMetrics'>5. Evaluation Metrics</a><br>
<a href = '#Accuracy'>5.a. Accuracy</a><br>
<a href = '#PrecisionRecallF1'>5.b. Precision, Recall, F1</a><br>
<a href = '#AUCROC'>5.c. AUC-ROC</a><br>
<br>

<a id = 'Basics'>1.Basics</a><br>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

<a id = '#LoadData'>1. a. Load Data</a><br>

In [ ]:
#read the data

In [ ]:
input_data = pd.read_csv('../input/bank.csv')

In [ ]:
#kick start things - check the num rows and cols

In [ ]:
print(input_data.shape)

In [ ]:
input_data['deposit_y_n'] = input_data['deposit'].map({'yes':1, 'no':0})

<a id = '#Logistic'>3.a. 1st model - Logistic</a><br>

In [ ]:
#lets build a simple model - as is the custom we will use logistic regression

In [ ]:
#we define the input and output variables
output_var = 'deposit_y_n'

In [ ]:
#lets use only the numeric cols for input vars
input_vars = [col for col in input_data.columns if input_data[col].dtype != 'object']

In [ ]:
input_vars

In [ ]:
#the list of input vars also has our output var; lets remove it
input_vars.remove(output_var)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logit_regr1 = LogisticRegression()

In [ ]:
logit_regr1.fit(X=input_data[input_vars], y=input_data[output_var])

In [ ]:
#now we have created the model; how do we know how good it is? we check the accuracy of the prediction

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
preds = logit_regr1.predict(X=input_data[input_vars])

In [ ]:
preds

In [ ]:
preds.shape

In [ ]:
print(accuracy_score(y_pred=preds, y_true=input_data[output_var]))

In [ ]:
pred_probs = logit_regr1.predict_proba(X=input_data[input_vars])

In [ ]:
pred_probs.shape

In [ ]:
pred_probs

In [ ]:
pred_probs[:1]

In [ ]:
pred_probs_of1s = pred_probs[:,1]

In [ ]:
pred_probs_of1s

In [ ]:
np.sum(pred_probs_of1s > 0.5)

In [ ]:
np.sum(pred_probs_of1s > 0.6)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
for i in range(10):
    threshold = i /10.0
    pred_class_new_threshold = [1 if x > threshold else 0 for x in pred_probs_of1s]
    print("for threshold of {}, precision: {} and recall: {}".format(threshold, 
                                                                     precision_score(y_true=input_data[output_var], y_pred=pred_class_new_threshold),
                                                                     recall_score(y_true=input_data[output_var], y_pred=pred_class_new_threshold)
                                                                    ))

In [ ]:
#lets go more granular for the threshold
precision_score_list = []
recall_score_list = []
for i in range(100):
    threshold = i /100.0
    pred_class_new_threshold = [1 if x > threshold else 0 for x in pred_probs_of1s]
    #print("for threshold of {}, precision: {} and recall: {}".format(threshold, 
    #                                                                 precision_score(y_true=input_data[output_var], y_pred=pred_class_new_threshold),
    #                                                                 recall_score(y_true=input_data[output_var], y_pred=pred_class_new_threshold)
    #                                                                ))
    
    precision_score_list.append(precision_score(y_true=input_data[output_var], y_pred=pred_class_new_threshold))
    recall_score_list.append(recall_score(y_true=input_data[output_var], y_pred=pred_class_new_threshold))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(x=precision_score_list, y=recall_score_list)
plt.xlabel('precision')
plt.ylabel('recall')
plt.show()

<a id = '#RandomForest'>3.b. Random Forest</a><br>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random_forest1 = RandomForestClassifier(n_estimators=5, max_depth=5)

In [ ]:
random_forest1.fit(X=input_data[input_vars], y=input_data[output_var])

In [ ]:
#lets get the accuracy
print(accuracy_score(y_pred=random_forest1.predict(X=input_data[input_vars]), y_true=input_data[output_var]))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_pred=random_forest1.predict(X=input_data[input_vars]), y_true=input_data[output_var])

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
random_forest1.predict_proba(input_data[input_vars])[:, 1]

In [ ]:
rf_1s_preds = random_forest1.predict_proba(input_data[input_vars])[ :, 1]

In [ ]:
fpr, tpr, threshold = roc_curve(y_true=input_data[output_var], y_score=rf_1s_preds)
roc_auc = auc(fpr, tpr)


In [ ]:
roc_auc

In [ ]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#enhance the plot
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#lets go more granular for the threshold
precision_score_list = []
recall_score_list = []
for i in range(100):
    threshold = i /100.0
    
    pred_class_new_threshold = [1 if x > threshold else 0 for x in rf_1s_preds]
    
    precision_score_list.append(precision_score(y_true=input_data[output_var], y_pred=pred_class_new_threshold))
    recall_score_list.append(recall_score(y_true=input_data[output_var], y_pred=pred_class_new_threshold))

In [ ]:
plt.scatter(x=precision_score_list, y=recall_score_list)
plt.xlabel('precision')
plt.ylabel('recall')
plt.show()

In [ ]:
fpr = 1 -  np.array(precision_score_list)

In [ ]:
plt.scatter(x=fpr, y=recall_score_list)

plt.xlabel('fpr')
plt.ylabel('recall')
plt.show()

In [ ]:
#AUC as the probability that a positive class would be ranked higher than a negative classs

In [ ]:
pos = rf_1s_preds[input_data[output_var] == 1]

In [ ]:
pos.shape

In [ ]:
neg = rf_1s_preds[input_data[output_var] == 0]
neg.shape

In [ ]:
np.random.choice(pos, size=1)[0]

In [ ]:
np.random.choice(neg, size=1)[0]

In [ ]:
pos_neg_list = []
for i in range(100000):
    pos_neg_list.append(np.random.choice(pos, size=1)[0] > np.random.choice(neg, size=1)[0])

In [ ]:
np.mean(pos_neg_list)

# Understanding AUC-ROC

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 

In [ ]:
Image(url='https://cdn-images-1.medium.com/max/800/1*Uu-t4pOotRQFoyrfqEvIEg.png')

In [ ]:
Image(url='https://cdn-images-1.medium.com/max/400/1*HmVIhSKznoW8tFsCLeQjRw.png')

In [ ]:
Image(url='https://cdn-images-1.medium.com/max/800/1*yF8hvKR9eNfqqej2JnVKzg.png')

In [ ]:
Image(url='https://cdn-images-1.medium.com/max/400/1*-tPXUvvNIZDbqXP0qqYNuQ.png')

In [ ]:
Image(url='https://cdn-images-1.medium.com/max/800/1*iLW_BrJZRI0UZSflfMrmZQ.png')

In [ ]:
Image(url='https://cdn-images-1.medium.com/max/400/1*k_MPO2Q9bLNH9k4Wlk6v_g.png')

In [ ]:
Image(url='https://cdn-images-1.medium.com/max/800/1*aUZ7H-Lw74KSucoLlj1pgw.png')

In [ ]:
Image(url='https://cdn-images-1.medium.com/max/400/1*H7JGQbaa06BUab6tvGNZKg.png')

In [ ]:
#in summary, higher the AUC, better the model